# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [22]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [23]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [24]:
year: int = 2020
topic: str = "FARMACIA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [25]:
add_list: list = [x for x in range(14, 67, 1)] + [x for x in range(104, 131, 1)] + [x for x in range(158, 162, 1)] + [x for x in range(167, 179, 1)]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [26]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. Las ortopramidas son un grupo de antipsicót...
3,se caracterizan estructuralmente por ser:
4,
5,1. Benzamidas con un grupo metoxilo en orto.
6,2. Benzoatos con un grupo metoxilo en orto.
7,3. Benzamidas con un grupo amino en orto.
8,4. Benzoatos con un grupo amino en orto.
9,


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [27]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,1,38,3,75,4.0,112,1,149,2
1,2,3,39,2,76,3.0,113,2,150,1
2,3,3,40,3,77,1.0,114,4,151,3
3,4,2,41,4,78,1.0,115,2,152,2
4,5,3,42,2,79,4.0,116,4,153,3
5,6,1,43,2,80,2.0,117,4,154,3
6,7,3,44,4,81,1.0,118,2,155,2
7,8,3,45,2,82,4.0,119,1,156,3
8,9,4,46,3,83,2.0,120,2,157,1
9,10,1,47,4,84,4.0,121,1,158,1


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [28]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [29]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [30]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [31]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [32]:
raw_exam_df.head(10)

0    1. Las ortopramidas son un grupo de antipsicót...
1            se caracterizan estructuralmente por ser:
2         1. Benzamidas con un grupo metoxilo en orto.
3          2. Benzoatos con un grupo metoxilo en orto.
4            3. Benzamidas con un grupo amino en orto.
5             4. Benzoatos con un grupo amino en orto.
6    2. Las moléculas que tienen propiedades física...
7    micas semejantes y que producen efectos fisiol...
8                             similares, se denominan:
9                           1. Isómeros estructurales.
Name: text, dtype: object

In [33]:
raw_exam_df.tail(10)

1576                                              alfa-2.
1577         4. Fisostigmina/inhibidor de colinesterasas.
1578    185. Señale la respuesta correcta con respecto...
1579     tóxicos conocidos como “bifenilos policlorados”:
1580    1. Son contaminantes con escasa persistencia e...
1581                                      medio ambiente.
1582             2. Actúan como disruptores estrogénicos.
1583    3. En su mayoría son sustancias de origen natu...
1584    4. No se conocen efectos de estos compuestos a...
1585                             de la glándula tiroides.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [34]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [35]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Las ortopramidas son un grupo de antipsicót...
1         1. Benzamidas con un grupo metoxilo en orto.
2          2. Benzoatos con un grupo metoxilo en orto.
3            3. Benzamidas con un grupo amino en orto.
4             4. Benzoatos con un grupo amino en orto.
5    2. Las moléculas que tienen propiedades física...
6                           1. Isómeros estructurales.
7                        2. Isómeros conformacionales.
8                                     3. Bioisósteros.
9                                     4. Enantiómeros.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [36]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 3
Number of rows not ending with '.', ':' or '?' = 3
[91, 338, 818]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [37]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [38]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [39]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    # rows_fix: list[int] = [90, 337, 817]
    # exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [40]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [41]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [42]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [43]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [44]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [45]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Las ortopramidas son un grupo de antipsicót...,1. Benzamidas con un grupo metoxilo en orto.,2. Benzoatos con un grupo metoxilo en orto.,3. Benzamidas con un grupo amino en orto.,4. Benzoatos con un grupo amino en orto.
1,2. Las moléculas que tienen propiedades física...,1. Isómeros estructurales.,2. Isómeros conformacionales.,3. Bioisósteros.,4. Enantiómeros.
2,3. ¿Cuál es la especie reactiva que se forma d...,1. Radical hidroxilo.,2. Catión aziridinio.,3. Catión metildiazonio.,4. Un grupo imino.
3,4. ¿Qué es el suxametonio (succinilcolina) res...,1. Un profármaco.,2. Un análogo blando.,3. Un antagonista competitivo.,4. Un análogo más lipófilo.
4,"5. El captopril, primer inhibidor comercializa...",1. No es importante para la actividad del capt...,2. Un grupo carboxílico interacciona mejor con...,3. Su presencia se asocia con efectos secundar...,4. Porque se mejoran las propiedades organolép...
5,"6. El cloroformiato de bencilo, también denomi...",1. Proteger un grupo amino.,2. Proteger un grupo carboxílico.,3. Activar un grupo carboxílico.,4. Acoplar un grupo amino con un grupo carboxí...
6,7. Señale la respuesta correcta en relación co...,1. Presentan un sustituyente pequeño sobre el ...,2. El grupo arilo debe ser un ciclo condensado...,3. Presentan un sustituyente en posición para ...,4. La amina debe ser terciaria.
7,8. ¿Qué requisito estructural debe estar prese...,1. Un grupo éster en la posición 17 beta.,2. Un átomo de flúor en la posición 9 alfa.,3. Un sustituyente en la posición 17 alfa.,4. Un heteroátomo en la posición 2.
8,9. ¿Cuál de las siguientes relaciones estructu...,1. La presencia de un grupo ciclopropilmetilo ...,2. La N-fenetilnormorfina tiene una acción ana...,3. La alquilación del hidroxilo fenólico reduc...,4. La reducción del doble enlace reduce la act...
9,10. ¿Cuál de los siguientes compuestos puede c...,1. Omeprazol.,2. Tolazamida.,3. Fentanilo.,4. Salbutamol.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [46]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [47]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [48]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [49]:
answers_df_clean.head(10)

,V,RC
0,1,1.0
1,2,3.0
2,3,3.0
3,4,2.0
4,5,3.0
5,6,1.0
6,7,3.0
7,8,3.0
8,9,4.0
9,10,1.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [50]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Las ortopramidas son un grupo de antipsicót...,1. Benzamidas con un grupo metoxilo en orto.,2. Benzoatos con un grupo metoxilo en orto.,3. Benzamidas con un grupo amino en orto.,4. Benzoatos con un grupo amino en orto.,1.0
1,2. Las moléculas que tienen propiedades física...,1. Isómeros estructurales.,2. Isómeros conformacionales.,3. Bioisósteros.,4. Enantiómeros.,3.0
2,3. ¿Cuál es la especie reactiva que se forma d...,1. Radical hidroxilo.,2. Catión aziridinio.,3. Catión metildiazonio.,4. Un grupo imino.,3.0
3,4. ¿Qué es el suxametonio (succinilcolina) res...,1. Un profármaco.,2. Un análogo blando.,3. Un antagonista competitivo.,4. Un análogo más lipófilo.,2.0
4,"5. El captopril, primer inhibidor comercializa...",1. No es importante para la actividad del capt...,2. Un grupo carboxílico interacciona mejor con...,3. Su presencia se asocia con efectos secundar...,4. Porque se mejoran las propiedades organolép...,3.0
5,"6. El cloroformiato de bencilo, también denomi...",1. Proteger un grupo amino.,2. Proteger un grupo carboxílico.,3. Activar un grupo carboxílico.,4. Acoplar un grupo amino con un grupo carboxí...,1.0
6,7. Señale la respuesta correcta en relación co...,1. Presentan un sustituyente pequeño sobre el ...,2. El grupo arilo debe ser un ciclo condensado...,3. Presentan un sustituyente en posición para ...,4. La amina debe ser terciaria.,3.0
7,8. ¿Qué requisito estructural debe estar prese...,1. Un grupo éster en la posición 17 beta.,2. Un átomo de flúor en la posición 9 alfa.,3. Un sustituyente en la posición 17 alfa.,4. Un heteroátomo en la posición 2.,3.0
8,9. ¿Cuál de las siguientes relaciones estructu...,1. La presencia de un grupo ciclopropilmetilo ...,2. La N-fenetilnormorfina tiene una acción ana...,3. La alquilación del hidroxilo fenólico reduc...,4. La reducción del doble enlace reduce la act...,4.0
9,10. ¿Cuál de los siguientes compuestos puede c...,1. Omeprazol.,2. Tolazamida.,3. Fentanilo.,4. Salbutamol.,1.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [51]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [52]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [53]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [54]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Las ortopramidas son un grupo de antipsicót...,1. Benzamidas con un grupo metoxilo en orto.,2. Benzoatos con un grupo metoxilo en orto.,3. Benzamidas con un grupo amino en orto.,4. Benzoatos con un grupo amino en orto.,1,2020
1,2. Las moléculas que tienen propiedades física...,1. Isómeros estructurales.,2. Isómeros conformacionales.,3. Bioisósteros.,4. Enantiómeros.,3,2020
2,3. ¿Cuál es la especie reactiva que se forma d...,1. Radical hidroxilo.,2. Catión aziridinio.,3. Catión metildiazonio.,4. Un grupo imino.,3,2020
3,4. ¿Qué es el suxametonio (succinilcolina) res...,1. Un profármaco.,2. Un análogo blando.,3. Un antagonista competitivo.,4. Un análogo más lipófilo.,2,2020
4,"5. El captopril, primer inhibidor comercializa...",1. No es importante para la actividad del capt...,2. Un grupo carboxílico interacciona mejor con...,3. Su presencia se asocia con efectos secundar...,4. Porque se mejoran las propiedades organolép...,3,2020
5,"6. El cloroformiato de bencilo, también denomi...",1. Proteger un grupo amino.,2. Proteger un grupo carboxílico.,3. Activar un grupo carboxílico.,4. Acoplar un grupo amino con un grupo carboxí...,1,2020
6,7. Señale la respuesta correcta en relación co...,1. Presentan un sustituyente pequeño sobre el ...,2. El grupo arilo debe ser un ciclo condensado...,3. Presentan un sustituyente en posición para ...,4. La amina debe ser terciaria.,3,2020
7,8. ¿Qué requisito estructural debe estar prese...,1. Un grupo éster en la posición 17 beta.,2. Un átomo de flúor en la posición 9 alfa.,3. Un sustituyente en la posición 17 alfa.,4. Un heteroátomo en la posición 2.,3,2020
8,9. ¿Cuál de las siguientes relaciones estructu...,1. La presencia de un grupo ciclopropilmetilo ...,2. La N-fenetilnormorfina tiene una acción ana...,3. La alquilación del hidroxilo fenólico reduc...,4. La reducción del doble enlace reduce la act...,4,2020
9,10. ¿Cuál de los siguientes compuestos puede c...,1. Omeprazol.,2. Tolazamida.,3. Fentanilo.,4. Salbutamol.,1,2020


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [55]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 96
Number of rows in filtered df = 96


**2. Showing the filtered DataFrame**

Using head and tail

In [56]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,"14. Las siguientes especies son dermatofitos, ...",1. Microsporum canis.,2. Trichophyton rubrum.,3. Trichosporon asahii.,4. Epidermophyton floccosum.,3,2020
1,15. El biomarcador 1-3-beta-D-glucano es útil ...,1. Aspergillus terreus.,2. Candida albicans.,3. Cryptococcus neoformans.,4. Pneumocystis jirovecii.,3,2020
2,"16. En el cultivo de un exudado ótico, tras 24...",1. Klebsiella pneumoniae.,2. Pseudomonas aeruginosa.,3. Haemophilus influenzae.,4. Staphylococcus aureus.,3,2020
3,17. ¿Cuál es el mecanismo de resistencia a qui...,1. Mutaciones en el gen de la ADN girasa (gyrA).,2. Mutaciones en el gen de la ARN polimerasa (...,3. Modificación de la diana por metilación del...,4. Beta-lactamasas de clase C de Ambler.,1,2020
4,18. ¿Qué característica de las siguientes NO e...,"1. Puede causar neumonía, meningitis, bacterie...",2. En la tinción de Gram se observan diplococo...,3. La colistina es el fármaco de elección para...,4. La detección de antígeno polisacárido C de ...,3,2020
5,19. Señale la opción FALSA respecto a Legionella:,1. Legionella pneumophila serogrupo 1 causa má...,2. La infección se adquiere por inhalación de ...,3. Levofloxacino y azitromicina son fármacos d...,4. La fiebre de Pontiac causada por Legionella...,4,2020
6,20. ¿Qué opción considera correcta en relación...,1. Nesseria gonorrhoeae es un patógeno exclusi...,2. La presencia de diplococos grampositivos en...,"3. En función de su cápsula polisacarídica, se...",4. Neisseria subflava es un patógeno de la oro...,1,2020
7,21. El agente etiológico más frecuente como ca...,1. Streptococcus dysgalactiae.,2. Estreptococo beta-hemolítico grupo G.,3. Arcanobacterium haemolyticum.,4. Streptococcus pyogenes.,4,2020
8,22. ¿Cuál es la respuesta correcta con respect...,1. Para el diagnóstico de enfermedad pulmonar ...,2. La reactividad de la prueba cutánea de la t...,3. La tinción microscópica de Ziehl-Neelsen ut...,4. Se puede medir la liberación de IFN-γ produ...,1,2020
9,23. ¿Cuál es la respuesta correcta con respect...,1. Es intracelular obligado.,2. Es muy sensible a altas concentraciones de ...,"3. Los neonatos, ancianos y mujeres gestantes ...",4. El tratamiento de elección de la enfermedad...,3,2020


In [57]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
86,169. ¿Dónde se produce la activación de los li...,"1. En los ganglios linfáticos, a través del co...",2. En cualquier sitio donde se encuentren con ...,3. En cualquier sitio donde encuentren una cél...,"4. En el sitio de infección, al entrar en cont...",1,2020
87,170. Los antígenos multivalentes de origen mic...,1. Linfocito B (BCR) de señales proporcionadas...,2. Linfocito B (BCR) y de señales proporcionad...,3. Linfocito T (TCR) y de señales proporcionad...,4. Linfocito B (BCR) y de citoquinas proporcio...,2,2020
88,171. Las células B se activan y se diferencian...,1. El timo.,2. Los ganglios linfáticos.,3. La medula ósea.,4. El momento en que el antígeno se une al TCR...,2,2020
89,"172. Los linfocitos T con TCR alfa/beta, CD4+ ...",1. Células: T CD8+ - TCRalfa/beta. Ligandos: p...,2. Células: T CD4+ reguladoras - TCRalfa/beta....,3. Células: T gamma-delta – TCRgamma/delta. Li...,4. Células: NKT - TCRalfa/beta. Ligandos: lípi...,4,2020
90,173. Se entiende por opsonización:,1. El mecanismo de la inmunidad natural de lis...,2. La fijación de proteínas del complemento y/...,3. La fijación de citocinas en la superficie d...,4. La interacción de anticuerpos con los recep...,2,2020
91,174. ¿Cuál de las siguientes relaciones entre ...,"1. TCR-CD4 - péptido/MHC II, primera señal de ...","2. TCR-CD8 - péptido/MHC I, primera señal de a...","3. CTLA4 - CD80/86, segunda señal de activació...","4. CD40 - CD40L, segunda señal de activación d...",3,2020
92,175. El procesamiento general de los antígenos...,1. Después de la presentación por las APC.,2. Por todas las células nucleadas del organismo.,3. En los compartimentos endosomales con pH bajo.,4. En forma de péptidos por enzimas proteolíti...,3,2020
93,176. La susceptibilidad a desarrollar una reac...,1. El repertorio de linfocitos B del progenito...,"2. Se heredan las moléculas del MHC, que son l...",3. El repertorio de linfocitos B en individuos...,"4. En respuesta al alérgeno, los individuos at...",4,2020
94,177. Señale cuál de estas afirmaciones NO es c...,1. Se utiliza para comprobar relaciones lineales.,2. Se utiliza para comparar variables cuantita...,3. Está muy afectado por los valores anormalme...,4. Se calcula a partir de ordenaciones de valo...,3,2020
95,178. ¿Cuál de las siguientes afirmaciones sobr...,1. Se conoce también como enfermedad de Carrión.,2. Es una enfermedad endémica en la cuenca med...,3. El tratamiento de elección es doxiciclina c...,4. Es una enfermedad exclusivamente humana.,2,2020


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [58]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [59]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [60]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [61]:

with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()